2項モデル（二項格子モデル）は、オプション価格付けの基本的な手法であり、マルチンゲール測度（リスク中立確率）を考慮することで、リスク中立確率のもとでの価格付けが可能になります。

以下のPythonコードでは、CRR（Cox-Ross-Rubinstein）二項モデルを構築し、マルチンゲール測度を用いてオプションの価格を計算します。

In [7]:
import numpy as np

def binomial_option_pricing(S0, K, T, r, sigma, N, option_type, american):
    dt = T / N  # 各ステップの時間間隔
    u = np.exp(sigma * np.sqrt(dt))  # 上昇率
    d = 1 / u  # 下降率
    p_rn = (np.exp(r * dt) - d) / (u - d)  # リスク中立確率
    
    # 株価のツリーを作成
    stock_tree = np.zeros((N + 1, N + 1))
    for i in range(N + 1):
        for j in range(i + 1):
            stock_tree[j, i] = S0 * (u ** (i - j)) * (d ** j)
    
    # オプション価値のツリーを作成
    option_tree = np.zeros((N + 1, N + 1))
    if option_type == "call":
        option_tree[:, N] = np.maximum(stock_tree[:, N] - K, 0)
    elif option_type == "put":
        option_tree[:, N] = np.maximum(K - stock_tree[:, N], 0)
    
    # バックワード・インダクションによる価格計算
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            discounted_value = np.exp(-r * dt) * (p_rn * option_tree[j, i + 1] + (1 - p_rn) * option_tree[j + 1, i + 1])
            if american:
                if option_type == "call":
                    exercise_value = np.maximum(stock_tree[j, i] - K, 0)
                else:  # put option
                    exercise_value = np.maximum(K - stock_tree[j, i], 0)
                option_tree[j, i] = np.maximum(discounted_value, exercise_value)
            else:
                option_tree[j, i] = discounted_value
    
    return option_tree[0, 0]

# パラメータ設定
S0 = 100   # 初期株価
K = 150    # 権利行使価格
T = 1      # 満期（年）
r = 0.05   # リスクフリーレート
sigma = 0.2  # ボラティリティ
N = 100    # 時間ステップ数

# コールオプションの価格を計算（アメリカンオプション）
american_put_price = binomial_option_pricing(S0, K, T, r, sigma, N, option_type="put", american=True)
print(f"American Put Option Price: {american_put_price:.4f}")

# コールオプションの価格を計算（ヨーロピアンオプション）
european_put_price = binomial_option_pricing(S0, K, T, r, sigma, N, option_type="put", american=False)
print(f"European Put Option Price: {european_put_price:.4f}")


American Put Option Price: 50.0000
European Put Option Price: 43.0347


このコードは、マルチンゲール測度（リスク中立確率）を考慮してヨーロピアン・コールオプションの価格を計算するものです。p_rn はリスク中立確率として計算され、各ノードでのオプション価値を後ろ向きに伝播することで、初期時点でのオプション価格を求めます。

このモデルをカスタマイズしたい場合は、条件やオプションの種類（アメリカンオプションなど）を拡張できます。

✅ 理論価格の算出

指定したパラメータ（$S_0$, $K$, $T$, $r$, $\sigma$,N）に基づいて、リスク中立確率のもとでオプションの理論価格を計算します。
これはブラック・ショールズモデルと同様に、アービトラージのない市場を前提としています。

✅ マルチンゲール性の考慮

リスク中立確率を計算し、それに基づいて価格をバックワード・インダクションで求めています。
これにより、マルチンゲール測度のもとでの価格付けが実装されています。

# 何ができないのか？
❌ 市場の実際の価格が適正かどうかの判断

このコードは理論価格を計算するだけであり、実際の市場価格と比較する部分はありません。
実際の市場データ（市場でのオプション価格やヒストリカルボラティリティ）を取得して比較する必要があります。
❌ ボラティリティの推定

インプライド・ボラティリティ（IV）の算出はしていません。
実際の市場価格がわかっている場合、数値的にIVを求めて適正価格をチェックすることが可能です（ニュートン法などを使用）。

# 市場価格と比較するには？

理論価格と市場価格を比較することで、オプションが割高か割安かを分析できます。

例えば、市場価格を取得（APIやデータソースから）理論価格と比較

- 市場価格 > 理論価格 → 割高（売り戦略を検討）
- 市場価格 < 理論価格 → 割安（買い戦略を検討）
 
このような分析を加えると、実際に市場で使える戦略が立てられます。

# 仕様変更

### アメリカンオプションの早期行使判定を追加

american という引数を追加し、アメリカンオプションの場合のみ 早期行使を考慮するようにした。

価格更新時に 「継続価値」 vs 「早期行使価値」 を比較し、大きい方を選択することで早期行使を可能にした。

デフォルトではヨーロピアンオプションを計算し、アメリカンオプションを使いたいときは american=True にする仕様に変更。

✅ 効果: 柔軟にヨーロピアン・アメリカン両方のオプションを計算できるようになった。